# Theme

This project acts as an AI-powered shopping assistant that understands user needs, retrieves similar smartphones from a dataset using embeddings, filters them by budget/brand/specifications, and finally uses an LLM to explain and recommend the best phone.

# Install

sentence-transformers → Converts product descriptions into vectors so similar phones can be retrieved intelligently.

transformers → Loads the FLAN-T5 model used for generating explanations and pros/cons of phones.

gradio → Creates the chatbot-style shopping interface.

torch → Runs model inference efficiently on CPU.

In [ ]:
!pip install -q sentence-transformers transformers gradio torch

Imports & Model Loading Cell → Loads all libraries and initializes the embedding model and language model used in the system.

In [ ]:
import os
import pandas as pd
import numpy as np
import gradio as gr
from sentence_transformers import SentenceTransformer, util
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# device
device = torch.device("cpu")

# Embedding model (fast for CPU)
EMBED_MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embed_model = SentenceTransformer(EMBED_MODEL_NAME)

# LLM (lightweight)
LLM_MODEL = "google/flan-t5-small"
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
llm_model = AutoModelForSeq2SeqLM.from_pretrained(LLM_MODEL).to(device)

# Path to your real dataset (already uploaded)
DATA_PATH = "/mnt/data/Flipkart_Mobiles.csv"


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Loads Python libraries.

Loads the embedding model (all-MiniLM-L6-v2)

Loads the LLM (flan-t5-small)

for DATASET use this below

URL: https://www.kaggle.com/datasets/huebitsvizg/flipkart-mobile-dataset

Download the CSV file to your computer.

Open Google Drive in your browser.

Click “New” on the left side.

Select “File upload”.

Choose the CSV file from your computer.

Wait for the upload to finish.

# Load dataset & basic preprocessing

In [ ]:
# Load CSV
df = pd.read_csv("/content/Flipkart_Mobiles.csv")

# Inspect top rows & columns
print("Loaded rows:", len(df))
print("Columns:", df.columns.tolist())
df.head()


Loaded rows: 3114
Columns: ['Brand', 'Model', 'Color', 'Memory', 'Storage', 'Rating', 'Selling Price', 'Original Price']


,Brand,Model,Color,Memory,Storage,Rating,Selling Price,Original Price
0,OPPO,A53,Moonlight Black,4 GB,64 GB,4.5,11990,15990
1,OPPO,A53,Mint Cream,4 GB,64 GB,4.5,11990,15990
2,OPPO,A53,Moonlight Black,6 GB,128 GB,4.3,13990,17990
3,OPPO,A53,Mint Cream,6 GB,128 GB,4.3,13990,17990
4,OPPO,A53,Electric Black,4 GB,64 GB,4.5,11990,15990


Reads your Flipkart CSV file.

Prints number of rows and column names.

Shows a preview of first few rows.

It was stored in colab context file

# Normalize & prepare text field for embeddings

In [ ]:
# Make sure expected columns exist
# Columns: ['Brand','Model','Color','Memory','Storage','Rating','Selling Price','Original Price']
required = ["Brand","Model","Color","Memory","Storage","Rating","Selling Price","Original Price"]
missing = [c for c in required if c not in df.columns]
if missing:
    raise ValueError(f"Missing columns in CSV: {missing}")

# Create a unified text field for embedding & display
def build_text_row(r):
    parts = [
        str(r.get("Brand","")).strip(),
        str(r.get("Model","")).strip(),
        f"Color: {r.get('Color','')}",
        f"Memory: {r.get('Memory','')}",
        f"Storage: {r.get('Storage','')}",
        f"Price: {r.get('Selling Price','')}",
    ]
    return ". ".join([p for p in parts if p])

df["text"] = df.apply(build_text_row, axis=1)
df["Selling Price"] = pd.to_numeric(df["Selling Price"], errors="coerce").fillna(0)
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce").fillna(0.0)


Makes sure all required columns exist (Brand, Model, Storage, etc.)

Creates one combined text per phone (brand + model + specs).

Converts price and rating to numbers.

# Precompute embeddings for catalog

In [ ]:
# Compute embeddings (this may take a few seconds)
catalog_texts = df["text"].tolist()
catalog_embeddings = embed_model.encode(catalog_texts, convert_to_tensor=True)
print("Embeddings shape:", catalog_embeddings.shape)


Embeddings shape: torch.Size([3114, 384])


Converts every product into a vector (embedding).

This allows your AI to find similar phones quickly.

retrieve_candidates() Cell → Retrieves the top phones semantically related to the user's natural-language request.

filter_and_rank() Cell → Applies filters like budget, brand, storage, and rating, then scores and ranks phones for best fit.

run_llm() Cell → Uses FLAN-T5 to turn the ranked list into human-friendly explanations and recommendations.

In [ ]:
def retrieve_candidates(query, top_k=10):
    q_emb = embed_model.encode(query, convert_to_tensor=True)
    scores = util.cos_sim(q_emb, catalog_embeddings).cpu().numpy().flatten()
    df_local = df.copy()
    df_local["sim_score"] = scores
    df_local = df_local.sort_values("sim_score", ascending=False).head(top_k)
    return df_local

def filter_and_rank(df_candidates, budget=None, brand=None, storage=None, min_rating=None, top_k=5):
    d = df_candidates.copy()
    if brand:
        d = d[d["Brand"].str.contains(str(brand), case=False, na=False)]
    if storage:
        d = d[d["Storage"].astype(str).str.contains(str(storage), case=False, na=False)]
    if budget and budget > 0:
        d = d[d["Selling Price"] <= budget]
    if min_rating and min_rating > 0:
        d = d[d["Rating"] >= float(min_rating)]
    if d.empty:
        return d
    # scoring: combine sim_score and normalized rating & price preference
    d["rating_norm"] = (d["Rating"] - d["Rating"].min()) / (d["Rating"].max() - d["Rating"].min() + 1e-9)
    # price preference: prefer lower price when query mentions budget — here normalize inverse price
    d["price_norm_inv"] = 1 - (d["Selling Price"] - d["Selling Price"].min()) / (d["Selling Price"].max() - d["Selling Price"].min() + 1e-9)
    d["final_score"] = d["sim_score"] * 0.6 + d["rating_norm"] * 0.25 + d["price_norm_inv"] * 0.15
    return d.sort_values("final_score", ascending=False).head(top_k)

def run_llm(prompt, max_out=180):
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=512).to(device)
    outputs = llm_model.generate(**inputs, max_length=max_out, min_length=20, no_repeat_ngram_size=2)
    return tokenizer.decode(outputs[0], skip_special_tokens=True).strip()


explain_recommendations() Cell → Builds a prompt summarizing product specs and asks the LLM to list pros, cons, and the best choice.

In [ ]:
def explain_recommendations(user_query, candidates_df):
    if candidates_df.empty:
        return "No products match your query/filters. Try broadening filters or removing budget constraints."
    prompt = f"User request: {user_query}\n\nProducts:\n"
    for idx, r in candidates_df.iterrows():
        prompt += f"- {r['Brand']} {r['Model']} | Price: ₹{int(r['Selling Price'])} | Rating: {r['Rating']}\n  {r['text']}\n"
    prompt += "\nFor each product, give 2 short pros and 1 short con. Then recommend the best product for the user's request with a one-line reason."
    return run_llm(prompt, max_out=350)


shopping_chat() Cell → Main handler that combines retrieval + ranking + LLM reasoning and returns formatted recommendations.

follow_up() Cell → Answers follow-up questions based only on previously suggested products, avoiding hallucination.

In [ ]:
# Maintain simple conversation context (last user query + last top-k text)
CHAT_CONTEXT = {"last_text": ""}

def shopping_chat(user_text, budget, brand, storage, min_rating):
    # Retrieve & filter
    retrieved = retrieve_candidates(user_text, top_k=50)
    ranked = filter_and_rank(retrieved, budget=budget, brand=brand, storage=storage, min_rating=min_rating, top_k=5)
    CHAT_CONTEXT["last_text"] = "\n".join(ranked["text"].astype(str).tolist())
    explanation = explain_recommendations(user_text, ranked)
    # candidates display markdown
    if not ranked.empty:
        lines = []
        for _, row in ranked.iterrows():
            lines.append(f"**{row['Brand']} {row['Model']}** — ₹{int(row['Selling Price'])} • Rating: {row['Rating']}\n\n{row['text']}\n")
        candidates_md = "\n---\n".join(lines)
    else:
        candidates_md = "No matching products found."
    return explanation, candidates_md

def follow_up(question):
    if not CHAT_CONTEXT.get("last_text"):
        return "Please ask about products first so I have context."
    prompt = f"Based only on the following product details:\n\n{CHAT_CONTEXT['last_text']}\n\nAnswer: {question}"
    return run_llm(prompt, max_out=200)


# Gradio UI (Chat + Filters + Results)

What it creates:

Chat input box

Budget, brand, storage, rating filters

Recommendation button

Follow-up question box

Product display area

In [ ]:
with gr.Blocks(title="Shopping GPT — Chat + Filters (Flipkart)") as demo:
    gr.Markdown("# 🛍️ Shopping GPT — Chat + Filters\nAsk in plain language and use the filters to refine results.")
    with gr.Row():
        with gr.Column(scale=2):
            user_input = gr.Textbox(label="Ask (e.g., 'Best phone under 15000 for camera')", lines=2, placeholder="Type your shopping request...")
            budget_in = gr.Number(label="Budget (INR, optional)", value=0)
            brand_in = gr.Textbox(label="Preferred Brand (optional)", placeholder="Samsung, Redmi, etc.")
            storage_in = gr.Textbox(label="Desired Storage (optional)", placeholder="64GB, 128GB")
            rating_in = gr.Slider(minimum=0, maximum=5, step=0.1, label="Minimum Rating (optional)", value=0)
            ask_btn = gr.Button("🔍 Recommend")
            gr.Markdown("### Follow-up question (after recommendations):")
            follow_q = gr.Textbox(label="Ask a follow-up question", lines=2)
            follow_btn = gr.Button("Ask")
        with gr.Column(scale=3):
            output_md = gr.Markdown("### Recommendations & Explanation will appear here")
            candidates_md = gr.Markdown("### Candidate products will appear here")

    ask_btn.click(fn=shopping_chat, inputs=[user_input, budget_in, brand_in, storage_in, rating_in], outputs=[output_md, candidates_md])
    follow_btn.click(fn=follow_up, inputs=follow_q, outputs=output_md)

demo.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b8636910a7c3884908.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
